# <center>OpenAI agent pattern: basic agent</center>

A starter guide for building a basic agent with tool calling using the `openai-agents` library. 

Here we've setup a basic agent that can answer questions about stocks using `web_search`. 

### Install Libraries

In [ ]:
# Install base libraries for OpenAI
!pip install -q openai openai-agents

# Install optional libraries for OpenInference/OpenTelemetry tracing
!pip install -q arize-phoenix-otel openinference-instrumentation-openai-agents openinference-instrumentation-openai

### Setup Keys

Add your OpenAI API key to the environment variable `OPENAI_API_KEY`.

Copy your Phoenix `API_KEY` from your settings page at [app.phoenix.arize.com](https://app.phoenix.arize.com).

In [ ]:
import os
from getpass import getpass

os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

OPENAI_API_KEY = globals().get("OPENAI_API_KEY") or getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### Setup Tracing

In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="openai-agents",
    endpoint="https://app.phoenix.arize.com/v1/traces",
    auto_instrument=True,
)

## Create your basic agent

In [ ]:
from agents import Agent, Runner, WebSearchTool

agent = Agent(
    name="Finance Agent",
    instructions="You are a finance agent that can answer questions about stocks. Use web search to retrieve up‑to‑date context. Then, return a brief, concise answer that is one sentence long.",
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
)

In [ ]:
from pprint import pprint

result = await Runner.run(agent, "what is the latest news on Apple?")

# Get the final output
print(result.final_output)

# Get the entire list of messages recorded to generate the final output
pprint(result.to_input_list())